In [1]:
import pandas as pd
import numpy as np
import scipy as sp 

from sklearn import preprocessing, linear_model
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns
from sklearn import svm
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# The Data from : https://www.sharecast.com/index/SP_500/prices/download

# The data path of input file
data_path = "./train.csv"
data_path2 = "./test.csv"

# Read csv file to pandas dataframe
pd_data = pd.read_csv(data_path)
pd_data2 = pd.read_csv(data_path2)

In [3]:
pd_data.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume
0,02-Jan-2009,902.99,931.80,934.73,899.35,4048270080
1,05-Jan-2009,929.17,927.45,936.63,919.53,5413910016
2,06-Jan-2009,931.17,934.70,943.85,927.28,5392620032
3,07-Jan-2009,927.45,906.65,927.45,902.37,4704940032
4,08-Jan-2009,905.73,909.73,910.00,896.81,4991549952


In [4]:
# 資料前處理

# 取出 close price
closeprice = pd_data['Close Price']
closeprice2 = pd_data2['Close Price']
pd_data = pd_data.drop(['Close Price', 'Date'], axis=1)
pd_data2 = pd_data2.drop(['Close Price', 'Date'], axis=1)

# 刪除最最最後一天的資料
pd_data = pd_data.drop(pd_data.index[[len(closeprice) - 1]])
pd_data2 = pd_data2.drop(pd_data2.index[[len(closeprice2) - 1]])

# 看出每一天與隔天的漲跌, 存到 y, y2 list 中
y = []
y2 = []

for i in range(1, len(closeprice)):
    # 漲
    if closeprice[i]>closeprice[i-1]:
        y.append(1)
    # 跌
    else:
        y.append(0)

for i in range(1, len(closeprice2)):
    # 漲
    if closeprice2[i]>closeprice2[i-1]:
        y2.append(1)
    # 跌
    else:
        y2.append(0)

pd_data.head()

,Open Price,High Price,Low Price,Volume
0,902.99,934.73,899.35,4048270080
1,929.17,936.63,919.53,5413910016
2,931.17,943.85,927.28,5392620032
3,927.45,927.45,902.37,4704940032
4,905.73,910.00,896.81,4991549952


In [13]:
# Logistic Regression

logistic_regr = linear_model.LogisticRegression()
logistic_regr.fit(pd_data, y)

# 計算準確率
predictions = logistic_regr.predict(pd_data)
accuracy = logistic_regr.score(pd_data, y)
accuracy2 = logistic_regr.score(pd_data2, y2)
print('(train data) Accuracy of Logistic Regression is ' + str(accuracy))
print('(test data) Accuracy of Logistic Regression is ' + str(accuracy2))

(train data) Accuracy of Logistic Regression is 0.5461776403004861
(test data) Accuracy of Logistic Regression is 0.5219123505976095


In [14]:
# SVM

clf = svm.SVC(C=0.8, kernel='rbf', gamma=20, decision_function_shape='ovo')
clf.fit(pd_data, y)

print ('(train data) Accuracy of SVM is ' + str(clf.score(pd_data, y)))
print ('(test data) Accuracy of SVM is ' + str(clf.score(pd_data2, y2)))

(train data) Accuracy of SVM is 1.0
(test data) Accuracy of SVM is 0.5219123505976095


In [7]:
# NN

# 定義模型
model = Sequential()
model.add(Dense(12, input_dim=4, init='uniform', activation='relu'))
model.add(Dense(4, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

# 編譯模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型
model.fit(pd_data, y, nb_epoch=150, batch_size=10, verbose=0)

# 評估模型
scores = model.evaluate(pd_data, y)
print("train data : %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# 評估模型
scores = model.evaluate(pd_data2, y2)
print("test data : %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/r-ni/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=4, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/home/r-ni/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="relu", kernel_initializer="uniform")`
  """
/home/r-ni/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  
/home/r-ni/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


2263/2263 [==============================] - 0s 16us/step
train data : acc: 54.62%
251/251 [==============================] - 0s 12us/step
test data : acc: 52.19%
